# Use scikit-learn to predict diabetes progression with Watson Machine Learning REST API

This notebook contains steps and code to demonstrate support of external machine learning models in Watson Machine Learning Service. This notebook introduces `cURL` calls for publishing, deploying (Web Service) and scoring ML scikit model.

Some familiarity with cURL is helpful. This notebook uses cURL examples.

This example was made based on `diabetes` dataset and trained scikit ML model that could be found inside the same repository under as follows: 
- `/data/diabetes/`
- `/models/scikit/diabetes/`


## Learning goals

The learning goals of this notebook are:

-  Working with Watson Machine Learning repository, deployment and scoring.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)  
2.	[WML Repository](#wml_repository)   
3.	[Model Deployment and Scoring](#deploy_and_score)   
4.  [Persist new version of the model](#update_model)
5.  [Redeploy and score new version of the model](#redeploy)
6.	[Cleaning](#cleaning)  
7.	[Summary](#summary)  

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).

You can find your COS credentials in COS instance dashboard under the **Service credentials** tab.
Go to the **Endpoint** tab in the COS instance's dashboard to get the endpoint information.

Authenticate the Watson Machine Learning service on IBM Cloud.

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.  

**NOTE:** You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.  



In [ ]:
%env API_KEY=...
%env WML_ENDPOINT_URL=...
%env WML_INSTANCE_CRN="fill out only if you want to create a new space"
%env WML_INSTANCE_NAME=...

%env COS_CRN="fill out only if you want to create a new space"
%env COS_ENDPOINT=...
%env COS_BUCKET=...
%env COS_ACCESS_KEY_ID=...
%env COS_SECRET_ACCESS_KEY=...
%env COS_API_KEY=...

%env SPACE_ID="fill out only if you have space already created"

%env DATAPLATFORM_URL=https://api.dataplatform.cloud.ibm.com   
%env AUTH_ENDPOINT=https://iam.cloud.ibm.com/oidc/token

<a id="wml_token"></a>
### Getting WML authorization token for further cURL calls

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-token" target="_blank" rel="noopener no referrer">Example of cURL call to get WML token</a>

In [2]:
%%bash --out token

curl -sk -X POST \
    --header "Content-Type: application/x-www-form-urlencoded" \
    --header "Accept: application/json" \
    --data-urlencode "grant_type=urn:ibm:params:oauth:grant-type:apikey" \
    --data-urlencode "apikey=$API_KEY" \
    "$AUTH_ENDPOINT" \
    | cut -d '"' -f 4

In [3]:
%env TOKEN=$token

env: TOKEN=eyJraWQiOiIyMDIwMDgyMjE2NTYiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJJQk1pZC01NTAwMDY3TkpEIiwiaWQiOiJJQk1pZC01NTAwMDY3TkpEIiwicmVhbG1pZCI6IklCTWlkIiwiaWRlbnRpZmllciI6IjU1MDAwNjdOSkQiLCJnaXZlbl9uYW1lIjoiSmFuIiwiZmFtaWx5X25hbWUiOiJTb2x0eXNpayIsIm5hbWUiOiJKYW4gU29sdHlzaWsiLCJlbWFpbCI6Ikphbi5Tb2x0eXNpa0BpYm0uY29tIiwic3ViIjoiSmFuLlNvbHR5c2lrQGlibS5jb20iLCJhY2NvdW50Ijp7InZhbGlkIjp0cnVlLCJic3MiOiIxY2Y5ZjBlYjZhZjQ0NTlmOTQ3ODcxYmQzM2I0NzMxOCIsImZyb3plbiI6dHJ1ZX0sImlhdCI6MTU5ODQyNjAxOCwiZXhwIjoxNTk4NDI5NjE4LCJpc3MiOiJodHRwczovL2lhbS50ZXN0LmNsb3VkLmlibS5jb20vb2lkYy90b2tlbiIsImdyYW50X3R5cGUiOiJ1cm46aWJtOnBhcmFtczpvYXV0aDpncmFudC10eXBlOmFwaWtleSIsInNjb3BlIjoiaWJtIG9wZW5pZCIsImNsaWVudF9pZCI6ImRlZmF1bHQiLCJhY3IiOjEsImFtciI6WyJwd2QiXX0.kell6Hbi6Wtr6T1szuMyFnirOQWu6eenGB8iGWy8kvQQM6dsafLOpKMVtDwVK2KdKp18zbzYozVV08OIfMHT8EwsUVO3DzzIyYRWB3v3-4okRhB8Uyt3kkyrVavYn6elowQihyIO1psQnurN4daqajhTPdJWJ4nLWSwAtMMUweium7cGjJlT3GVaYLidN7Mx8atZKWqqGTFWvy5hVPWT_b8RmGQmssH3mAfvb-frjjoXnuhz0VFeaYPHRFpSzIR3dUZLw041

<a id="space_creation"></a>
### Space creation
**Tip:** If you do not have `space` already created, please convert below three cells to `code` and run them.

First of all, you need to create a `space` that will be used in all of your further cURL calls. 
If you do not have `space` already created, below is the cURL call to create one.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_create" 
target="_blank" rel="noopener no referrer">Space creation</a>

Space creation is asynchronous. This means that you need to check space creation status after creation call.
Make sure that your newly created space is `active`.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_get" 
target="_blank" rel="noopener no referrer">Get space information</a>

---

<a id="wml_repository"></a>
## 2. Manage WML Repository

In this section you will learn how to upload your ML model, list your models stored in repository, delete your model and update it.

<a id="model_storing"></a>
### Model storing
Store information about your model to WML repository.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_create" 
target="_blank" rel="noopener no referrer">Model storing</a>

In [8]:
%%bash --out model_payload

MODEL_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "scikit_diabetes_model","description": "This is description","type": "scikit-learn_0.20", "software_spec": {"name": "scikit-learn_0.20-py3.6"}}'
echo $MODEL_PAYLOAD | python -m json.tool

In [18]:
%env MODEL_PAYLOAD=$model_payload

env: MODEL_PAYLOAD={
    "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750",
    "name": "scikit_diabetes_model",
    "description": "This is description",
    "type": "scikit-learn_0.20",
    "software_spec": {
        "name": "scikit-learn_0.20-py3.6"
    }
}


In [20]:
%%bash --out model_id -s "$model_payload"

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/models?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [21]:
%env MODEL_ID=$model_id

env: MODEL_ID=0517d315-171a-4d36-9e98-8f8571ed268e


### Create model revision for further update

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_create_revision" 
target="_blank" rel="noopener no referrer">Model revision</a>

In [23]:
%%bash --out revision_payload

REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Initial model."}'
echo $REVISION_PAYLOAD | python -m json.tool

In [24]:
%env REVISION_PAYLOAD=$revision_payload

env: REVISION_PAYLOAD={
    "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750",
    "commit_message": "Initial model."
}


In [25]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REVISION_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/revisions?version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "software_spec": {
            "id": "09c5a1d0-9c1e-4473-a344-eb7b665ff687",
            "name": "scikit-learn_0.20-py3.6"
        },
        "type": "scikit-learn_0.20"
    },
    "metadata": {
        "commit_info": {
            "commit_message": "Initial model.",
            "committed_at": "2020-08-26T07:22:45.002Z"
        },
        "created_at": "2020-08-26T07:20:14.498Z",
        "description": "This is description",
        "id": "0517d315-171a-4d36-9e98-8f8571ed268e",
        "modified_at": "2020-08-26T07:20:14.498Z",
        "name": "scikit_diabetes_model",
        "owner": "IBMid-5500067NJD",
        "rev": "1",
        "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750"
    }
}


<a id="content_upload"></a>
### Model content upload
Now you need to upload your model content into the WML repository.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_upload_content" 
target="_blank" rel="noopener no referrer">Upload model content</a>

In [26]:
%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@../../../../models/scikit/diabetes/model/diabetes_model.tar.gz" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2020-08-01&content_format=native" \
    | grep "attachment_id" | awk -F '"' '{ print $4 }'

In [27]:
%env ATTACHMENT_ID=$attachment_id

env: ATTACHMENT_ID=50d9c2fc-2795-4d72-9abe-c6de818084bc


<a id="model_download"></a>
### Download model
If you want to download your saved model, please make the following call.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_filtered_download" 
target="_blank" rel="noopener no referrer">Download model content</a>

In [28]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --output "model.tar.gz" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/download?space_id=$SPACE_ID&version=2020-08-01"

In [29]:
!ls -l model.tar.gz

-rw-r--r--  1 jansoltysik  staff  765 Aug 26 09:23 model.tar.gz


<a id="deploy_and_score"></a>
## 3. Deploy and Score

In this section you will learn how to deploy and score pipeline model as webservice using WML instance.

<a id="deployment_creation"></a>
### Deployment creation
This example uses scikit-learn model deployment and `S` hardware specification.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_create" 
target="_blank" rel="noopener no referrer">Create deployment</a>

In [30]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "Diabetes deployment", "description": "This is description","online": {},"hardware_spec": {"name": "S"},"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD | python -m json.tool 

In [31]:
%env DEPLOYMENT_PAYLOAD=$deployment_payload

env: DEPLOYMENT_PAYLOAD={
    "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750",
    "name": "Diabetes deployment",
    "description": "This is description",
    "online": {},
    "hardware_spec": {
        "name": "S"
    },
    "asset": {
        "id": "0517d315-171a-4d36-9e98-8f8571ed268e"
    }
}


In [32]:
%%bash --out deployment_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DEPLOYMENT_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/deployments?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p

In [33]:
%env DEPLOYMENT_ID=$deployment_id

env: DEPLOYMENT_ID=17419640-07c6-40a9-9361-54cb34f5f068


<a id="deployment_details"></a>
### Get deployment details
As deployment API is asynchronous, please make sure your deployment is in `ready` state before going to the next points.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_get" 
target="_blank" rel="noopener no referrer">Get deployment details</a>

In [34]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "asset": {
            "id": "0517d315-171a-4d36-9e98-8f8571ed268e"
        },
        "custom": {},
        "description": "This is description",
        "hardware_spec": {
            "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
            "name": "S",
            "num_nodes": 1
        },
        "name": "Diabetes deployment",
        "online": {},
        "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750",
        "status": {
            "online_url": {
                "url": "https://wml-fvt.ml.test.cloud.ibm.com/ml/v4/deployments/17419640-07c6-40a9-9361-54cb34f5f068/predictions"
            },
            "state": "initializing"
        }
    },
    "metadata": {
        "created_at": "2020-08-26T07:24:50.832Z",
        "description": "This is description",
        "id": "17419640-07c6-40a9-9361-54cb34f5f068",
        "modified_at": "2020-08-26T07:24:50.832Z",
        "name": "Diabetes deployment",
        "owner": "IBMid-5500067NJD",
        "space_id

<a id="webservice_score"></a>
### Scoring of a webservice
If you want to make a `score` call on your deployment, please follow a below method:

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_compute_predictions" 
target="_blank" rel="noopener no referrer">Score your deployment</a>

In [35]:
%%bash --out scoring_payload

SCORING_PAYLOAD='{"space_id": "$SPACE_ID","input_data": [{"fields": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"], "values": [[-0.00188201652779104, -0.044641636506989, -0.0514740612388061, -0.0263278347173518, -0.00844872411121698, -0.019163339748222, 0.0744115640787594, -0.0394933828740919, -0.0683297436244215, -0.09220404962683], [0.0852989062966783, 0.0506801187398187, 0.0444512133365941, -0.00567061055493425, -0.0455994512826475, -0.0341944659141195, -0.0323559322397657, -0.00259226199818282, 0.00286377051894013, -0.0259303389894746]]}]}'
echo $SCORING_PAYLOAD | python -m json.tool

In [36]:
%env SCORING_PAYLOAD=$scoring_payload

env: SCORING_PAYLOAD={
    "space_id": "$SPACE_ID",
    "input_data": [
        {
            "fields": [
                "age",
                "sex",
                "bmi",
                "bp",
                "s1",
                "s2",
                "s3",
                "s4",
                "s5",
                "s6"
            ],
            "values": [
                [
                    -0.00188201652779104,
                    -0.044641636506989,
                    -0.0514740612388061,
                    -0.0263278347173518,
                    -0.00844872411121698,
                    -0.019163339748222,
                    0.0744115640787594,
                    -0.0394933828740919,
                    -0.0683297436244215,
                    -0.09220404962683
                ],
                [
                    0.0852989062966783,
                    0.0506801187398187,
                    0.0444512133365941,
                    -0.00567061055493425,
          

In [39]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$SCORING_PAYLOAD"\
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2020-08-01" \
    | python -m json.tool

{
    "predictions": [
        {
            "fields": [
                "prediction"
            ],
            "values": [
                [
                    68.44064739029959
                ],
                [
                    178.20406005411206
                ]
            ]
        }
    ]
}


<a id="deployments_list"></a>
### Listing all deployments

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_list" 
target="_blank" rel="noopener no referrer">List deployments details</a>

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployments?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="update_model"></a>
## 4. Persist new version of the model

In this section, you'll learn how to store new version of your model in Watson Machine Learning repository.

### Model update
Below you can find how ML model can be updated with new version on WML repository.

List model revisions.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_list_revisions" 
target="_blank" rel="noopener no referrer">List revisions</a>

In [40]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/revisions?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "first": {
        "href": "/ml/v4/models/0517d315-171a-4d36-9e98-8f8571ed268e/revisions?space_id=74133c06-dce2-4dfc-b913-2e0dc8efc750&limit=100"
    },
    "limit": 100,
    "resources": [
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Initial model.",
                    "committed_at": "2020-08-26T07:22:45.002Z"
                },
                "created_at": "2020-08-26T07:20:14.498Z",
                "description": "This is description",
                "id": "0517d315-171a-4d36-9e98-8f8571ed268e",
                "modified_at": "2020-08-26T07:20:14.498Z",
                "name": "scikit_diabetes_model",
                "owner": "IBMid-5500067NJD",
                "rev": "1",
                "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750"
            }
        }
    ]
}


### Create second model revision

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_create_revision" 
target="_blank" rel="noopener no referrer">Create model revision</a>

In [41]:
%%bash --out revision_payload

REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Updated model."}'
echo $REVISION_PAYLOAD | python -m json.tool

In [42]:
%env REVISION_PAYLOAD=$revision_payload

env: REVISION_PAYLOAD={
    "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750",
    "commit_message": "Updated model."
}


In [43]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REVISION_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/revisions?version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "software_spec": {
            "id": "09c5a1d0-9c1e-4473-a344-eb7b665ff687",
            "name": "scikit-learn_0.20-py3.6"
        },
        "type": "scikit-learn_0.20"
    },
    "metadata": {
        "commit_info": {
            "commit_message": "Updated model.",
            "committed_at": "2020-08-26T07:27:58.002Z"
        },
        "created_at": "2020-08-26T07:20:14.498Z",
        "description": "This is description",
        "id": "0517d315-171a-4d36-9e98-8f8571ed268e",
        "modified_at": "2020-08-26T07:22:57.103Z",
        "name": "scikit_diabetes_model",
        "owner": "IBMid-5500067NJD",
        "rev": "2",
        "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750"
    }
}


### Update model metadata  
  
For example update  model name or description

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_update" 
target="_blank" rel="noopener no referrer">Patch model</a>

In [45]:
%%bash --out update_payload

echo '[{"op": "add", "path": "/name", "value": "updated scikit model"}]' | python -m json.tool

In [46]:
%env UPDATE_PAYLOAD=$update_payload

env: UPDATE_PAYLOAD=[
    {
        "op": "add",
        "path": "/name",
        "value": "updated scikit model"
    }
]


In [47]:
%%bash

curl -sk -X PATCH \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$UPDATE_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "software_spec": {
            "id": "09c5a1d0-9c1e-4473-a344-eb7b665ff687",
            "name": "scikit-learn_0.20-py3.6"
        },
        "type": "scikit-learn_0.20"
    },
    "metadata": {
        "commit_info": {
            "committed_at": "2020-08-26T07:30:00.776Z"
        },
        "created_at": "2020-08-26T07:20:14.498Z",
        "description": "This is description",
        "id": "0517d315-171a-4d36-9e98-8f8571ed268e",
        "modified_at": "2020-08-26T07:29:57.625Z",
        "name": "updated scikit model",
        "owner": "IBMid-5500067NJD",
        "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750"
    }
}


### Upload new model content

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_upload_content" 
target="_blank" rel="noopener no referrer">Upload model content</a>

In [48]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@../../../../models/scikit/diabetes/model/new_diabetes_model.tar.gz" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2020-08-01&content_format=native" \
    | python -m json.tool

{
    "attachment_id": "67bc6365-ba71-4aa1-a8a8-08ea2a6643db",
    "content_format": "native",
    "persisted": true
}


#### List model revisions to see a new one just created

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_list_revisions" 
target="_blank" rel="noopener no referrer">List revisions</a>

In [49]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/revisions?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "first": {
        "href": "/ml/v4/models/0517d315-171a-4d36-9e98-8f8571ed268e/revisions?space_id=74133c06-dce2-4dfc-b913-2e0dc8efc750&limit=100"
    },
    "limit": 100,
    "resources": [
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Updated model.",
                    "committed_at": "2020-08-26T07:27:58.002Z"
                },
                "created_at": "2020-08-26T07:20:14.498Z",
                "description": "This is description",
                "id": "0517d315-171a-4d36-9e98-8f8571ed268e",
                "modified_at": "2020-08-26T07:22:57.103Z",
                "name": "scikit_diabetes_model",
                "owner": "IBMid-5500067NJD",
                "rev": "2",
                "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750"
            }
        },
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Initial model.",
                    "comm

Now we have updated model content and model name in repository.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_get" 
target="_blank" rel="noopener no referrer">Get model details</a>

In [50]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "software_spec": {
            "id": "09c5a1d0-9c1e-4473-a344-eb7b665ff687",
            "name": "scikit-learn_0.20-py3.6"
        },
        "type": "scikit-learn_0.20"
    },
    "metadata": {
        "commit_info": {
            "committed_at": "2020-08-26T07:30:27.113Z"
        },
        "created_at": "2020-08-26T07:20:14.498Z",
        "description": "This is description",
        "id": "0517d315-171a-4d36-9e98-8f8571ed268e",
        "modified_at": "2020-08-26T07:30:22.119Z",
        "name": "updated scikit model",
        "owner": "IBMid-5500067NJD",
        "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750"
    }
}


<a id="redeploy"></a>
## 5. Redeploy and score new version of the model

Below you can see how deployment can be updated with new version of the model without any change for scoring url.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_update" 
target="_blank" rel="noopener no referrer">Deployment update</a>

In [51]:
%%bash --out redeploy_payload

echo '[{"op": "replace", "path": "/asset", "value": {"id": "'"$MODEL_ID"'"}}]' \
     | python -m json.tool

In [52]:
%env REDEPLOY_PAYLOAD=$redeploy_payload

env: REDEPLOY_PAYLOAD=[
    {
        "op": "replace",
        "path": "/asset",
        "value": {
            "id": "0517d315-171a-4d36-9e98-8f8571ed268e"
        }
    }
]


In [53]:
%%bash

curl -sk -X PATCH \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REDEPLOY_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "asset": {
            "id": "0517d315-171a-4d36-9e98-8f8571ed268e"
        },
        "custom": {},
        "description": "This is description",
        "hardware_spec": {
            "id": "Not_Applicable",
            "name": "S",
            "num_nodes": 1
        },
        "name": "Diabetes deployment",
        "online": {},
        "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750",
        "status": {
            "online_url": {
                "url": "https://wml-fvt.ml.test.cloud.ibm.com/ml/v4/deployments/17419640-07c6-40a9-9361-54cb34f5f068/predictions"
            },
            "state": "ready"
        }
    },
    "metadata": {
        "created_at": "2020-08-26T07:24:50.832Z",
        "description": "This is description",
        "id": "17419640-07c6-40a9-9361-54cb34f5f068",
        "modified_at": "2020-08-26T07:31:44.566Z",
        "name": "Diabetes deployment",
        "owner": "IBMid-5500067NJD",
        "space_id": "74133c06-dce2-4dfc-b913-2

### Score updated webservice

In [54]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$SCORING_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2020-08-01" \
    | python -m json.tool

{
    "predictions": [
        {
            "fields": [
                "prediction"
            ],
            "values": [
                [
                    68.44064739029959
                ],
                [
                    178.20406005411206
                ]
            ]
        }
    ]
}


<a id="cleaning"></a>
## 6. Cleaning section

Below section is useful when you want to clean all of your previous work within this notebook.
Just convert below cells into the `code` and run them.

<a id="deployment_delete"></a>
### Deleting deployment
**Tip:** You can delete existing deployment by calling DELETE method.

<a id="model_delete"></a>
### Delete model from repository
**Tip:** If you want to completely remove your stored model and model metadata, just use a DELETE method.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_delete" 
target="_blank" rel="noopener no referrer">Delete model from repository</a>

<a id="summary"></a>
## 7. Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use `cURL` calls to store, deploy and score a scikit-learn ML model in WML. 


### Authors

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM  
**Jan Sołtysik**, Intern in Watson Machine Learning at IBM

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.